<a href="https://colab.research.google.com/github/mrengineermso/Deep-Learning-Project/blob/main/new_animal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from cv2 import resize
from glob import glob

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
!ls '/content/gdrive/My Drive/Animal_Classification/'

animals  hayvansınıflandırma.ipynb  model_weight.h5


In [28]:
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/gdrive/My Drive/Animal_Classification/animals',
  validation_split=0.2,
  subset='training',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)



Found 5402 files belonging to 90 classes.
Using 4322 files for training.


In [29]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  '/content/gdrive/My Drive/Animal_Classification/animals',
  validation_split=0.2,
  subset='validation',
  image_size=(img_height, img_width),
  batch_size=32,
  seed=42,
  shuffle=True)

Found 5402 files belonging to 90 classes.
Using 1080 files for validation.


In [30]:
base_model = tf.keras.applications.VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(img_height, img_width, 3)
)
base_model.trainable = False 

In [31]:
inputs = tf.keras.Input(shape=(img_height, img_width, 3))
x = tf.keras.applications.vgg16.preprocess_input(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(90)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.__operators__.getitem (S  (None, 224, 224, 3)      0         
 licingOpLambda)                                                 
                                                                 
 tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)      0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d_4   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout (Dropout)           (None, 512)               0   

In [32]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
num_epochs = 10
history=model.fit(train_ds, validation_data=val_ds, epochs=num_epochs,
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            # Stop training when `val_loss` is no longer improving
            monitor="val_loss",
            # "no longer improving" being defined as "no better than 1e-2 less"
            min_delta=1e-2,
            # "no longer improving" being further defined as "for at least 2 epochs"
            patience=2,
            verbose=1,
            restore_best_weights=True
        )
    ]
)

Epoch 1/10
136/136 [==============================] - 34s 239ms/step - loss: 0.5605 - accuracy: 0.8635 - val_loss: 0.9535 - val_accuracy: 0.8222
Epoch 2/10
136/136 [==============================] - 33s 233ms/step - loss: 0.4639 - accuracy: 0.8758 - val_loss: 0.9851 - val_accuracy: 0.8241
Epoch 3/10
136/136 [==============================] - 33s 231ms/step - loss: 0.4114 - accuracy: 0.8954 - val_loss: 0.9799 - val_accuracy: 0.8176
Epoch 3: early stopping


In [42]:
# Modeli kaydetme
model.save('/content/gdrive/My Drive/Animal_Classification/model.h5')

# Epoch-accuracy, epoch-loss ve epoch-F1 skor grafiklerini çizdirme
epochs = range(1, num_epochs + 1)
plt.figure(figsize=(15, 5))



<Figure size 1500x500 with 0 Axes>

<Figure size 1500x500 with 0 Axes>

In [44]:
import matplotlib.pyplot as plt

